#### Procesamiento de datos y EDA

In [0]:
from pyspark.sql.functions import *

In [0]:
raw_covid = spark.read.csv("/mnt/coviddl1/raw/covid_population.csv", sep=',', header=True)

In [0]:
raw_covid.head(12)

In [0]:
from pyspark.sql.functions import *
raw_covid_sel = raw_covid.select(col("date"),
                                 col("state"),
                                 col("place_type").alias("place"),
                                 col("deaths"),
                                 col("estimated_population_2019").alias("population"),
                                 col("death_rate"))

raw_covid_sel.show()

In [0]:
raw_covid.createOrReplaceTempView("covid")

In [0]:
%sql
SELECT * FROM covid

In [0]:
%sql
DESCRIBE covid

In [0]:
%sql
SELECT date, state, deaths FROM covid WHERE state in ("MG", "RJ", "SP") and place_type = "state"

In [0]:
%sql
SELECT state, deaths FROM covid WHERE place_type = "state" AND is_last = "True" ORDER BY deaths DESC

In [0]:
%sql
SELECT date, deaths FROM covid WHERE state = "MG" and place_type = "state" ORDER BY date DESC

In [0]:
query = """
  SELECT date, deaths FROM covid WHERE state = "MG" and place_type = "state" ORDER BY date DESC
"""

df = spark.sql(query)
df = df.toPandas()

In [0]:
df_cov = spark.createDataFrame(df)

In [0]:
df_cov.write.format("com.databricks.spark.csv").option("header","true").option("delimiter", ",").mode("overwrite").save("/mnt/coviddl1/processed/population")

#### Machine Learning con Spark

In [0]:
import pandas as pd
import logging

logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j").setLevel(logging.ERROR)


query = """
  SELECT string(date) as ds, int(deaths) as y FROM covid WHERE state = "MG" and place_type = "state" order by date
"""

df = spark.sql(query)
df = df.toPandas()
display(df)

In [0]:
from prophet import Prophet

m = Prophet()
m.fit(df)

future = m.make_future_dataframe(periods=30)

forecast = m.predict(future)

fig1 = m.plot(forecast)

In [0]:
forecast.head()

In [0]:
forecast = spark.createDataFrame(forecast)

In [0]:
forecast.createOrReplaceTempView("forecast")

output = spark.sql("SELECT ds,yhat FROM forecast")

In [0]:
output.write.format("com.databricks.spark.csv").option("header","true").option("delimiter", ",").mode("overwrite").save("/mnt/coviddl1/processed/population")